Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [1]:
let name = "Sathvick reddy beeravalli"
let rollno = "CS19B010"

val name : string = "Sathvick reddy beeravalli"


val rollno : string = "CS19B010"


## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please recall the policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Mutability and Modules

In this assignment, you will design and implement a couple of mutable data structures and operations on them.

## Problem 1

Implement structures `IntShowable` and `FloatShowable` that satisfy the following signature. Use `string_of_int: int -> string` and `string_of_float: float -> string` for the corresponding `string_of_t` functions.

In [2]:
module type Showable = sig
  type t
  val string_of_t : t -> string
end

module type Showable = sig type t val string_of_t : t -> string end


In [3]:
module IntShowable = struct
  type t=int 
  let string_of_t x= string_of_int x
end  
module FloatShowable = struct
  type t=float 
  let string_of_t x= string_of_float x
end  


module IntShowable : sig type t = int val string_of_t : int -> string end


module FloatShowable :
  sig type t = float val string_of_t : float -> string end


In [4]:
(* 15 points *)
assert (IntShowable.string_of_t 10 = "10");
assert (FloatShowable.string_of_t 0.0 = "0.")

- : unit = ()


## Problem 2

Implement a functor 

```ocaml
module MakeNode : 
  functor (C : Showable) -> 
    DoublyLinkedListNode with type content = C.t
``` 

where `DoublyLinkedListNode` is the module type:

In [5]:
module type DoublyLinkedListNode = sig
  type t
  (** The type of doubly linked list node *)
  
  type content
  (** The type of content stored in the doubly linked list node *)

  val create : content -> t 
  (** create a new doubly linked list node with [content] as the content. 
      The next and previous nodes are [None]. *)
  
  val get_next : t -> t option
  (** [get_next t] returns [Some t'] if [t'] is the successor node of [t]. 
      If [t] has no successor, then return [None] *)
      
  val get_prev : t -> t option
  (** [get_prev t] returns [Some t'] if [t'] is the predecessor node of [t]. 
      If [t] has no predecessor, then return [None] *)
      
  val get_content : t -> content
  (** [content t] returns the content [c] of node [t] *)
      
  val set_next : t -> t option -> unit
  (** [set_next t t'] updates the next node of [t] to be [t'] *)
  
  val set_prev : t -> t option -> unit
  (** [set_prev t t'] updates the prev node of [t] to be [t'] *)
  
  val string_of_content : content -> string
  (** [string_of_content c] returns the string form of content [c] *)
end

module type DoublyLinkedListNode =
  sig
    type t
    type content
    val create : content -> t
    val get_next : t -> t option
    val get_prev : t -> t option
    val get_content : t -> content
    val set_next : t -> t option -> unit
    val set_prev : t -> t option -> unit
    val string_of_content : content -> string
  end


In [6]:
module MakeNode (C : Showable) : DoublyLinkedListNode with type content = C.t = struct

 type content = C.t
 type t = Cons of C.t * t option ref * t option ref
 (*value,next,prev*)
 let create x = Cons (x,ref None,ref None)
 let get_next x = match x with
                  |Cons(p,q,r)-> !q;;

 let get_prev x = match x with
                  |Cons(p,q,r)-> !r;;

 let get_content x = match x with 
                     |Cons(p,q,r) -> p;;
                     
 let set_next x y = match x with
                    |Cons(p,q,r) -> q:=y;;
                    
 let set_prev x y = match x with
                    |Cons(p,q,r) -> r:=y;;
 
 let string_of_content x = C.string_of_t x;;
 
end

module MakeNode :
  functor (C : Showable) ->
    sig
      type t
      type content = C.t
      val create : content -> t
      val get_next : t -> t option
      val get_prev : t -> t option
      val get_content : t -> content
      val set_next : t -> t option -> unit
      val set_prev : t -> t option -> unit
      val string_of_content : content -> string
    end


In [7]:
module IntNode = MakeNode(IntShowable)

module IntNode :
  sig
    type t = MakeNode(IntShowable).t
    type content = IntShowable.t
    val create : content -> t
    val get_next : t -> t option
    val get_prev : t -> t option
    val get_content : t -> content
    val set_next : t -> t option -> unit
    val set_prev : t -> t option -> unit
    val string_of_content : content -> string
  end


In [8]:
(* 15 points *)
let open IntNode in 
let i1 = create 1 in
assert (get_content i1 = 1)

- : unit = ()


In [9]:
(* 15 points *)
let open IntNode in 
let i1,i2,i3 = create 1, create 2, create 3 in
set_next i1 @@ Some i2;
set_next i2 @@ Some i3;
set_prev i2 @@ Some i1;
set_prev i3 @@ Some i2;
let _ = assert (match get_next i1 with None -> false | Some i2 -> get_content i2 = 2) in
()

- : unit = ()


## Problem 3

Implement a functor 

```ocaml 
module MakeList : 
  functor (N : DoublyLinkedListNode) -> 
    DoublyLinkedList with type node = N.t 
                      and type content = N.content
```

where the module type `DoublyLinkedList` is defined as follows:


In [10]:
module type DoublyLinkedList = sig
  type t 
  (** The type of doubly linked list *)
  
  type node
  (** The type of a doubly linked list node *)
  
  type content
  (** The type of content stored in the [node] of doubly linked list *)
    
  val create : unit -> t 
  (** Creates a new doubly linked list *)
  
  val assign : t -> node option -> unit
  (** [assign t n] makes the node as the head node of the list. 
      The original contents of the list are dropped. *)
  
  val t_of_list : content list -> t
  (** [t_of_list l] returns a new doubly linked list with the list of 
      elements from the list [l]. *)
  
  val is_empty : t -> bool
  (** [is_empty l] return true if [t] is empty *)
  
  val first : t -> node option
  (** [first l] return [Some n] if the list is non-empty. Otherwise, return [None] *)
  
  val insert_first : t -> content -> node
  (** [insert_first l c] inserts a new doubly linked list node [n] as the first 
      node in [l] with [c] as the content. Returns [n]. *)
      
  val insert_after : node -> content -> node
  (** [insert_after n c] inserts a new node [n'] with content [c] after the node [n].
      Returns [n']. *)
  
  val remove : t -> node -> unit
  (** [remove l n] removes the node [n] from the list [l] *)
  
  val iter : t -> (content -> unit) -> unit
  (** [iter l f] applies [f] to each element of the list in the list order. *)
  
  val iter_node : t -> (node -> unit) -> unit
  (** [iter_node l f] applies [f] to each node of the list in the list order. 
      [f] may delete the current node or insert a new node after the current node. 
      In either case, [iter_node] is applied to the rest of the original list. *)
      
  val string_of_t : t -> string
  (** [string_of_t (t_of_list [1;2;3])] returns the string "[1->2->3]".
      [string_of_t (t_of_list [])] returns the string "[]" *)
end

module type DoublyLinkedList =
  sig
    type t
    type node
    type content
    val create : unit -> t
    val assign : t -> node option -> unit
    val t_of_list : content list -> t
    val is_empty : t -> bool
    val first : t -> node option
    val insert_first : t -> content -> node
    val insert_after : node -> content -> node
    val remove : t -> node -> unit
    val iter : t -> (content -> unit) -> unit
    val iter_node : t -> (node -> unit) -> unit
    val string_of_t : t -> string
  end


In [11]:
(* Implement the functor MakeList *)


module MakeList (N : DoublyLinkedListNode) : DoublyLinkedList with type content = N.content 
                                                               and type node = N.t = 
struct
  type t = N.t option ref
  type node = N.t
  type content = N.content
  let create () = ref None
  let assign x y = x:=y
(*     *)

 let t_of_list l =
 
 let insert x y = 
  
  let n =  N.create y  in
  begin match !x with
  | Some z -> N.set_prev z (Some n) 
  | None -> ()
  end;
  N.set_next n !x ;
  x := Some n;
  x in 
 
  let rec f l acc = 
 match l with
 |x::xs->insert (f xs acc) x  
 |_-> acc
 in f l (ref None)
 
(*    *)
 
 let is_empty x = !x = None
  let first x = !x 

 let insert_first x y = 
  
  let n =  N.create y  in
  begin match !x with
  | Some z -> N.set_prev z (Some n) 
  | None -> ()
  end;
  N.set_next n !x ;
  x := Some n;
  n;;
  
  let insert_after x y =
  
  let n = N.create y in
   
   begin match N.get_next x with 
   | Some z -> N.set_prev z (Some n) ; N.set_next n (Some z)
   | None -> ()
   end;
   
   N.set_next x (Some n);
   N.set_prev n (Some x);
  n;;
  
   let remove l n =
    let x = N.get_prev n in
    let y = N.get_next n in
    begin match x with
    | Some z -> N.set_next z (y)
    | None -> l := y
    end;
    begin match y with
    | Some z -> N.set_prev z (x)
    | None -> ()
    end;
    N.set_prev n (None);
    N.set_next n (None);;
   
(*      *)
  let iter t f =
  
  let rec loop x = 
  begin match x with
    | None -> ()
    | Some z -> f (N.get_content z); loop (N.get_next z)
  end;
  in
  loop !t;;
   (*applied to each node*)
   
   let iter_node t f= 
  let rec loop x = 
  begin match x with
    | None -> ()
    | Some z -> loop (N.get_next z) ; f z
  end;
  in
  loop !t;;

    let string_of_t l =  
   let rec loop acc l = 
   begin match !l with
   |None -> acc
   |Some z -> if N.get_next z = None then acc ^ (N.string_of_content (N.get_content z))
                else loop (acc ^ (N.string_of_content (N.get_content z))^"->") (ref (N.get_next z))
   
   end;
   in "[" ^ (loop "" l) ^ "]"
end



module MakeList :
  functor (N : DoublyLinkedListNode) ->
    sig
      type t
      type node = N.t
      type content = N.content
      val create : unit -> t
      val assign : t -> node option -> unit
      val t_of_list : content list -> t
      val is_empty : t -> bool
      val first : t -> node option
      val insert_first : t -> content -> node
      val insert_after : node -> content -> node
      val remove : t -> node -> unit
      val iter : t -> (content -> unit) -> unit
      val iter_node : t -> (node -> unit) -> unit
      val string_of_t : t -> string
    end


In [12]:
(* 10 points *)
let module L = MakeList(IntNode) in

let open L in
assert (is_empty (t_of_list []));
let l = t_of_list [1;2;3] in
assert (not @@ is_empty l);
let n = match first l with 
| None -> failwith "impossible" 
| Some n -> n
in
assert (1 = IntNode.get_content n);
L.(assign l (first (t_of_list [4;5;6])));
match first l with
| None -> failwith "impossible"
| Some n -> assert (IntNode.get_content n = 4)


- : unit = ()


In [13]:
(* 10 points *)
let module L = MakeList(IntNode) in

let open L in
let l = t_of_list [1;2;3] in
let n1 = match first l with
  | Some n1 -> n1
  | None -> failwith "impossible"
in
let n2 = match IntNode.get_next n1 with
  | Some n2 -> n2
  | None -> failwith "impossible"
in
remove l n2;
let n3 = match IntNode.get_next n1 with
  | Some n3 -> n3
  | None -> failwith "impossible"
in
let _ = assert (IntNode.get_content n3 = 3) in
remove l n1;
match first l with
| None -> failwith "impossible"
| Some n3 -> assert (IntNode.get_content n3 = 3)


- : unit = ()


In [14]:
(* 15 points *)
let module L = MakeList(IntNode) in
let open L in
let l = t_of_list [1;2;3] in
let s = ref 0 in
iter l (fun c -> s := !s + c);
assert (!s = 6);
iter_node l (fun n -> if IntNode.get_content n mod 2 == 0 then remove l n);
let n1 = match first l with
  | None -> failwith "impossible"
  | Some n -> n
in
assert (IntNode.get_content n1 = 1)


- : unit = ()


## Problem 4

Implement the functor

```ocaml
module MakeDllFunctions :
  functor (N : DoublyLinkedListNode) ->
    Dll_functions with type node = N.t 
                   and type content = N.content 
                   and type dll = MakeList(N).t
```

where the module type `Dll_functions` is:

In [15]:
module type Dll_functions = sig
  type dll
  (** The type of doubly linked list *)
  
  type node
  (** The type of doubly linked list node *)
  
  type content
  (** The type of doubly linked list content *)

  val length : dll -> int
  (** Returns the length of the list. 
      Use [DLL.iter] to implement it. *)
  
  val duplicate : dll -> unit
  (** Given a doubly linked list [l] = [1->2], [duplicate l] returns [1->1->2->2]. 
      Use [DLL.iter_node] to implement it. *)
        
  val rotate : dll -> int -> unit
  (** [rotate l n] rotates the list by [n] nodes. 
      If the list is [1->2->3->4->5], then [rotate l 0] will not modify the list. 
      [rotate l 2] will modify the list to be [3->4->5->1->2]. 
      Assume that [0 <= n < length l]. *)
  
   val reverse : dll -> unit
   (** [reverse l] reverses the list *)
end

module type Dll_functions =
  sig
    type dll
    type node
    type content
    val length : dll -> int
    val duplicate : dll -> unit
    val rotate : dll -> int -> unit
    val reverse : dll -> unit
  end


In [16]:
module MakeDllFunctions (N : DoublyLinkedListNode) 
  : Dll_functions with type node = N.t 
                   and type content = N.content 
                   and type dll = MakeList(N).t = struct
   type dll = MakeList(N).t
   type node = N.t
   type content = N.content

   open MakeList(N)
   let f x lengt = let y= ref  lengt in iter x (fun t -> y:=!y+1); !y;; 
   let length l  = f l 0;;
  
  let insert_aft x y =
  
  let n = N.create y in
   
   begin match N.get_next x with 
   | Some z -> N.set_prev z (Some n) ; N.set_next n (Some z)
   | None -> ()
   end;
   
   N.set_next x (Some n);
   N.set_prev n (Some x);;

  let duplicate l = iter_node l (fun x -> insert_aft x (N.get_content x));;
   
   let rec fn l idx =
   match idx with 
   |0-> l
   |_->begin match l with
       |None-> None
       |Some x->fn (N.get_next x) (idx-1)
       end;;
       
   let rotate l idx= 
   let node = fn (first l) idx in
   let last = fn (first l) (length l -1) in
   begin match last with
   |None->()
   |Some n-> match (first l) with
             |None->()
             |Some y->N.set_next n (first l); N.set_prev y last;
   end;
   begin match node with
   |None->()
   |Some k->begin match N.get_prev k with
            |None->()
            |Some s->N.set_next s None ; N.set_prev k None 
             end;
   end;
   
    assign l node;; 

   let reverse l=
    let len=length l in
    
     iter_node l (fun n->match len with
                         |0->()
                         |_->match fn (first l) (length l -1) with
                             |None->()
                             |Some p-> insert_aft p (N.get_content n);remove l n);;
                         
                           
   
end

module MakeDllFunctions :
  functor (N : DoublyLinkedListNode) ->
    sig
      type dll = MakeList(N).t
      type node = N.t
      type content = N.content
      val length : dll -> int
      val duplicate : dll -> unit
      val rotate : dll -> int -> unit
      val reverse : dll -> unit
    end


In [17]:
(* 10 points *)
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
let _ = assert (F.length l = 3) in
F.duplicate l;
let _ = assert (F.length l = 6) in
()

- : unit = ()


In [18]:
(* 10 points *)
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
let _ = assert (F.length l = 3) in
F.duplicate l;
let _ = assert (F.length l = 6) in
let _ = assert ("[1->1->2->2->3->3]" = string_of_t l) in
F.rotate l 3;
let _ = assert ("[2->3->3->1->1->2]" = string_of_t l) in
F.rotate l 0;
let _ = assert ("[2->3->3->1->1->2]" = string_of_t l) in
let l = t_of_list [] in
F.rotate l 0;
let _ = assert ("[]" = string_of_t l) in
()

- : unit = ()


In [19]:
(* 10 points *)
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
F.reverse l;
assert ("[3->2->1]" = string_of_t l)


- : unit = ()


## Hash Table

Implement a hash table using [separate chaining](https://en.wikipedia.org/wiki/Hash_table#Separate_chaining_with_linked_lists). Use an OCaml array for the array of buckets. The documentation of OCaml arrays is found in the [OCaml manual](https://caml.inria.fr/pub/docs/manual-ocaml/libref/Array.html).

Each element in the array is a doubly linked list in order to allow chaining. Use your doubly linked list that you have implemented earlier. The hash table will be implemented as a functor that accepts two module arguments. The first one defines the type of the key, size of the array and the hash function. The signature of that module is:

In [20]:
module type Key = sig
  type t
  (** The type of key *)
  
  val num_buckets : int
  (** The number of buckets i.e, the size of the buckets array *)
  
  val hash : t -> int
  (** Hashes the key to an integer between [0, num_buckets) *)
  
  val string_of_t : t -> string
  (** Returns the string representation of key *)
end

module type Key =
  sig
    type t
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


The second functor argument defines the type of content and its signature is `Showable` that we had defined at the beginning of this assignment. 

The signature of the `Hash_table` module is:

In [21]:
module type Hash_table = sig
  type key
  (** The type of key *)
  
  type content
  (** The type of value *)
  
  type t
  (** The type of the hash table *)
  
  val create : unit -> t
  (** Creates a new hash table *)
  
  val put :  t -> key -> content -> unit
  (** [put h k v] associates the key [k] with value [v] in the hash table [h]. 
      If the binding for [k] already exists, overwrite it. *)

  val get : t -> key -> content option
  (** [get h k] returns [Some v], where [v] is the value associated with the 
      key [k] in the hash table [h]. Returns [None] if the [k] is not bound in [h]. *)
      
  val remove : t -> key -> unit
  (** [remove h k] removes the association for key [k] from the hash table [h] if it exists *)
  
  val length : t -> int
  (** Return the number of key-value pairs in the hash table *)
  
  val string_of_t : t -> string
  (** Returns a string version of the hash table. Can be in any format. Only for debugging. *)
  
  val chain_length : t -> int -> int
  (** [chain_length h bucket] returns the length of the doubly linked list 
      at the index [bucket] in the underlying array. 
      Assume that [0 <= bucket < length(bucket_array)] *)
end

module type Hash_table =
  sig
    type key
    type content
    type t
    val create : unit -> t
    val put : t -> key -> content -> unit
    val get : t -> key -> content option
    val remove : t -> key -> unit
    val length : t -> int
    val string_of_t : t -> string
    val chain_length : t -> int -> int
  end


## Problem 5

Implement the functor 

```ocaml
module Make_hash_table :
  functor (K : Key) (C : Showable) ->
    Hash_table with type key = K.t 
                and type content = C.t
```

In [22]:
module Make_hash_table (K : Key) (C : Showable):
    Hash_table  with type key = K.t 
                and type content = C.t = 
struct
type key= K.t;;
type content=C.t;;

module IntShow = struct
  type t={
          key:K.t;
          content:C.t;
          };; 
  let string_of_t x= (K.string_of_t x.key)^ "->:" ^ (C.string_of_t x.content)
end  
type t=MakeList(MakeNode(IntShow)).t array;;

open MakeNode(IntShow)
open MakeList(MakeNode(IntShow));;
let f x= create ();;

let create ()= Array.init (K.num_buckets) f ;;


 let rec get_k x ke =
 match x with
 |None->None
 |Some y-> let z=get_content y in 
            if z.key=ke then (Some (z.content)) 
            else get_k (get_next y) ke;;
 
 let get arr ke =
  let lis= Array.get arr (K.hash ke) in
  get_k (first lis) ke  ;;
  
 
(*   *)
 let rec remove_k x lis ke =
 match x with
 |None->()
 |Some y-> let z=get_content y in 
            if z.key=ke then (remove lis y) 
            else remove_k (get_next y) lis ke;;
 
  let remove arr ke=
  let lis= Array.get arr (K.hash ke) in
  remove_k (first lis) lis ke  ;;
(*    *)

let put arr ke con =
  remove arr ke;
 let lis= Array.get arr (K.hash ke) in
(*  let nod = ({ key = ke ; content = con}) in *)
  
 let sth = insert_first lis { key = ke ; content = con} in
  
assign lis (Some sth) ;;
 


 let f x lengt = let y= ref  lengt in iter x (fun t -> y:=!y+1); !y;; 
   let length_lis l  = f l 0;;

let rec total_len arr x acc=
match x with
|0->acc
|_->total_len arr (x-1) (acc+length_lis (Array.get arr (x-1))) 


let length arr =let x = Array.length arr in total_len arr x 0;;

 
    let s_of_t l =  
   let rec loop acc l = 
   begin match !l with
   |None -> acc
   |Some z -> if get_next z = None then acc ^ (string_of_content (get_content z))
                else loop (acc ^ (string_of_content (get_content z))^"->") (ref (get_next z))
   
   end;
   in "[" ^ (loop "" l) ^ "]"

(*  *)
let rec fn arr acc x  =
 match x with 
 |0->""
 |_-> let lis= (Array.get arr (x-1)) in fn arr (acc^(string_of_int x)^" : "^(string_of_t lis)) (x-1)
      
let string_of_t arr = let x = Array.length arr in
fn arr "" x;;
      
(*  *)

let chain_length arr ind = let lis =Array.get arr ind in length_lis lis;;
end

module Make_hash_table :
  functor (K : Key) (C : Showable) ->
    sig
      type key = K.t
      type content = C.t
      type t
      val create : unit -> t
      val put : t -> key -> content -> unit
      val get : t -> key -> content option
      val remove : t -> key -> unit
      val length : t -> int
      val string_of_t : t -> string
      val chain_length : t -> int -> int
    end


In [23]:
#show_module_type Key

module type Key =
  sig
    type t
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


In [24]:
module K : Key with type t = int = struct
  type t = int
  let num_buckets = 64
  let hash k = k mod num_buckets
  let string_of_t = string_of_int
end

module StringShowable : Showable with type t = string = struct
  type t = string
  let string_of_t s = s
end

module H = Make_hash_table(K)(StringShowable)

open H

module K :
  sig
    type t = int
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


module StringShowable : sig type t = string val string_of_t : t -> string end


module H :
  sig
    type key = K.t
    type content = StringShowable.t
    type t = Make_hash_table(K)(StringShowable).t
    val create : unit -> t
    val put : t -> key -> content -> unit
    val get : t -> key -> content option
    val remove : t -> key -> unit
    val length : t -> int
    val string_of_t : t -> string
    val chain_length : t -> int -> int
  end


In [25]:
(* 15 points *)
let h = create () in
put h 0 "zero";
put h 1 "one";
assert (get h 0 = Some "zero");
assert (get h 1 = Some "one");
put h 0 "-zero-";
assert (get h 0 = Some "-zero-")


- : unit = ()


In [26]:
(* 20 points *)
let h = create () in
put h 0 "zero";
put h 1 "one";
assert (get h 0 = Some "zero");
assert (get h 1 = Some "one");
put h 64 "-zero-";
assert (get h 0 = Some "zero");
assert (get h 64 = Some "-zero-");
assert (length h = 3);
assert (chain_length h 0 = 2);
assert (chain_length h 1 = 1)


- : unit = ()
